In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import torch
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout,Activation,MaxPooling2D,ReLU,Add,GlobalAvgPool2D
from tensorflow.keras import Model,Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping


2023-05-31 09:31:34.204196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get the Data and Labels

In [10]:
num_testsub = 32
num_classes = 4
num_channel = 32
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_stft/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [11]:
data_dict = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [13]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [14]:
valence.shape

(1280,)

In [12]:
data_dict['sub01'].shape

(40, 32, 44)

## Create train test split

### Data

In [15]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,32,44))
train[:] = np.nan
#train1 = 
test = np.zeros((0,32,44))
test[:] = np.nan

participation_counter = 0
for sub in data_dict.keys():
    if participation_counter < 22:
        train = np.concatenate((train, data_dict[sub]), axis=0)
    else:
        test = np.concatenate((test, data_dict[sub]), axis=0)
    participation_counter = participation_counter + 1

In [16]:
np.isnan(train).any()

False

In [17]:
np.isnan(test).any()

False

In [18]:
train.shape

(880, 32, 44)

In [19]:
test.shape

(400, 32, 44)

### Labels

In [25]:
val22 = valence[: 880]
aro22 = arousal[: 880]
dom22 = dominance[: 880]
lik22 = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

## KNN

In [20]:
train.reshape(-1, 32*44).shape

(880, 1408)

In [21]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [22]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [27]:
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
score = []


for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(train.reshape(-1, 32*44),traain,test.reshape(-1, 32*44),tesst,kval = k)
    scorelist.append((sc,k))
    score.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

K value = 1
######################################### Valence #########################################
Accuracy 49.25
######################################### Arousal #########################################
Accuracy 48.5
######################################### Dominance #########################################
Accuracy 53.5
######################################### Liking #########################################
Accuracy 53.5
K value = 2
######################################### Valence #########################################
Accuracy 42.0
######################################### Arousal #########################################
Accuracy 44.75
######################################### Dominance #########################################
Accuracy 41.0
######################################### Liking #########################################
Accuracy 47.75
K value = 3
######################################### Valence #########################################
Accuracy 48.25
####

Accuracy 49.0
######################################### Arousal #########################################
Accuracy 50.74999999999999
######################################### Dominance #########################################
Accuracy 60.0
######################################### Liking #########################################
Accuracy 61.0
K value = 21
######################################### Valence #########################################
Accuracy 52.5
######################################### Arousal #########################################
Accuracy 51.24999999999999
######################################### Dominance #########################################
Accuracy 63.5
######################################### Liking #########################################
Accuracy 61.5
K value = 22
######################################### Valence #########################################
Accuracy 49.0
######################################### Arousal ##################################

Accuracy 55.75
######################################### Dominance #########################################
Accuracy 68.0
######################################### Liking #########################################
Accuracy 62.25000000000001
K value = 40
######################################### Valence #########################################
Accuracy 53.25
######################################### Arousal #########################################
Accuracy 55.50000000000001
######################################### Dominance #########################################
Accuracy 64.75
######################################### Liking #########################################
Accuracy 62.0
K value = 41
######################################### Valence #########################################
Accuracy 54.25
######################################### Arousal #########################################
Accuracy 56.00000000000001
######################################### Dominance ###############

In [ ]:
len(scorelist)

In [28]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(56.49999999999999, 44)
(56.75, 43)
(69.75, 47)
(62.5, 33)


## SVM

In [29]:
# Define SVM based model for augmented data
def svmmodel(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

  # SVM model

  model = SVC(C=c,kernel=kernel,gamma = 'auto')
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [31]:
# Using actual data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(train.reshape(-1, 32*44),traain,test.reshape(-1, 32*44),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 56.0
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Acc

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 56.25 <br>
Accuracy of  Dominance 73.0 <br>
Accuracy of  Liking 62.50 <br>

### Neural Network

In [34]:
# Split the data into training/testing sets
def deepmodel(modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size):
  # ()
  
  ytrain = to_categorical(ytrain)
  ytest = to_categorical(ytest)



  input_shape = (32, 44, 1)
  # print(xtrain.shape,xtest.shape)
  xtrain = np.reshape(xtrain,[-1, 32, 44, 1])
  xtest = np.reshape(xtest,[-1, 32, 44, 1])
  print("xtrain.shape,ytrain.shape")
  print(xtrain.shape,ytrain.shape)
  if modelnum == 1:
  # Model 1
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(1, activation='sigmoid'))
  elif modelnum == 2:
    # Resnet kind of model

    def conv_bn_relu(x, filters, kernel_size, strides):
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


    def identity_block(tensor, filters):
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=1)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        x = ReLU()(x)
        return x


    def projection_block(tensor, filters, strides):
        # left branch
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        # right branch
        shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x


    def resnet_block(x, filters, reps, strides):
        x = projection_block(x, filters=filters, strides=strides)
        for _ in range(reps-1):  # the -1 is because the first block was a Conv one
            x = identity_block(x, filters=filters)
        return x


    input = Input(shape=(32, 44, 1))

    x = conv_bn_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)  # s=2 ([2]: conv3_1)
    x = resnet_block(x, filters=256, reps=6, strides=2)  # s=2 ([2]: conv4_1)
    x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)

    x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global

    # # output = Dense(1, activation='sigmoid')(x)
    output = Dense(2, activation='softmax')(x)
    model = Model(input, output)



  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(),
                    metrics=['accuracy'])
  # Fit data to model
  history = model.fit(xtrain,ytrain ,
            batch_size = batch_size,
            epochs=nepochs,
            verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(xtest, ytest, verbose=0)#verbose =1 
  y_pred = model.predict(xtest)
  #     print("Fold number",fold_no)
  acc = scores[1]*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print("Loss",scores[0])
  # print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  # print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  return acc

## Normal Data

In [36]:
# Using original data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,train.reshape(-1, 32,44),traain,test.reshape(-1, 32,44),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(880, 32, 44, 1) (880, 2)
Epoch 1/200
28/28 [==============================] - 2s 27ms/step - loss: 0.6980 - accuracy: 0.5295
Epoch 2/200
28/28 [==============================] - 1s 27ms/step - loss: 0.6880 - accuracy: 0.5352
Epoch 3/200
28/28 [==============================] - 1s 27ms/step - loss: 0.6774 - accuracy: 0.5591
Epoch 4/200
28/28 [==============================] - 1s 26ms/step - loss: 0.6655 - accuracy: 0.6068
Epoch 5/200
28/28 [==============================] - 1s 27ms/step - loss: 0.6457 - accuracy: 0.6534
Epoch 6/200
28/28 [==============================] - 1s 28ms/step - loss: 0.6293 - accuracy: 0.6534
Epoch 7/200
28/28 [==============================] - 1s 26ms/step - loss: 0.5936 - accuracy: 0.6818
Epoch 8/200
28/28 [==============================] - 1s 29ms/step - loss: 0.5522 - accuracy: 0.7216
Epoch 9/200
28/28 [==============================] - 1s 

28/28 [==============================] - 1s 31ms/step - loss: 4.9977e-05 - accuracy: 1.0000
Epoch 79/200
28/28 [==============================] - 1s 31ms/step - loss: 4.8720e-05 - accuracy: 1.0000
Epoch 80/200
28/28 [==============================] - 1s 30ms/step - loss: 4.6900e-05 - accuracy: 1.0000
Epoch 81/200
28/28 [==============================] - 1s 29ms/step - loss: 4.5520e-05 - accuracy: 1.0000
Epoch 82/200
28/28 [==============================] - 1s 29ms/step - loss: 4.3826e-05 - accuracy: 1.0000
Epoch 83/200
28/28 [==============================] - 1s 31ms/step - loss: 4.2551e-05 - accuracy: 1.0000
Epoch 84/200
28/28 [==============================] - 1s 28ms/step - loss: 4.1294e-05 - accuracy: 1.0000
Epoch 85/200
28/28 [==============================] - 1s 27ms/step - loss: 4.0094e-05 - accuracy: 1.0000
Epoch 86/200
28/28 [==============================] - 1s 28ms/step - loss: 3.8993e-05 - accuracy: 1.0000
Epoch 87/200
28/28 [==============================] - 1s 28ms/step -

28/28 [==============================] - 1s 29ms/step - loss: 7.3282e-06 - accuracy: 1.0000
Epoch 156/200
28/28 [==============================] - 1s 29ms/step - loss: 7.1890e-06 - accuracy: 1.0000
Epoch 157/200
28/28 [==============================] - 1s 31ms/step - loss: 7.0319e-06 - accuracy: 1.0000
Epoch 158/200
28/28 [==============================] - 1s 29ms/step - loss: 6.8962e-06 - accuracy: 1.0000
Epoch 159/200
28/28 [==============================] - 1s 30ms/step - loss: 6.7482e-06 - accuracy: 1.0000
Epoch 160/200
28/28 [==============================] - 1s 31ms/step - loss: 6.6080e-06 - accuracy: 1.0000
Epoch 161/200
28/28 [==============================] - 1s 29ms/step - loss: 6.4838e-06 - accuracy: 1.0000
Epoch 162/200
28/28 [==============================] - 1s 32ms/step - loss: 6.3647e-06 - accuracy: 1.0000
Epoch 163/200
28/28 [==============================] - 1s 30ms/step - loss: 6.2420e-06 - accuracy: 1.0000
Epoch 164/200
28/28 [==============================] - 1s 29

28/28 [==============================] - 1s 32ms/step - loss: 5.5012e-04 - accuracy: 1.0000
Epoch 32/200
28/28 [==============================] - 1s 31ms/step - loss: 5.0366e-04 - accuracy: 1.0000
Epoch 33/200
28/28 [==============================] - 1s 31ms/step - loss: 4.5857e-04 - accuracy: 1.0000
Epoch 34/200
28/28 [==============================] - 1s 30ms/step - loss: 4.1886e-04 - accuracy: 1.0000
Epoch 35/200
28/28 [==============================] - 1s 28ms/step - loss: 3.8507e-04 - accuracy: 1.0000
Epoch 36/200
28/28 [==============================] - 1s 32ms/step - loss: 3.5708e-04 - accuracy: 1.0000
Epoch 37/200
28/28 [==============================] - 1s 30ms/step - loss: 3.3477e-04 - accuracy: 1.0000
Epoch 38/200
28/28 [==============================] - 1s 30ms/step - loss: 3.1056e-04 - accuracy: 1.0000
Epoch 39/200
28/28 [==============================] - 1s 30ms/step - loss: 2.8950e-04 - accuracy: 1.0000
Epoch 40/200
28/28 [==============================] - 1s 30ms/step -

28/28 [==============================] - 1s 31ms/step - loss: 1.6441e-05 - accuracy: 1.0000
Epoch 110/200
28/28 [==============================] - 1s 30ms/step - loss: 1.5842e-05 - accuracy: 1.0000
Epoch 111/200
28/28 [==============================] - 1s 31ms/step - loss: 1.5290e-05 - accuracy: 1.0000
Epoch 112/200
28/28 [==============================] - 1s 31ms/step - loss: 1.4766e-05 - accuracy: 1.0000
Epoch 113/200
28/28 [==============================] - 1s 30ms/step - loss: 1.4298e-05 - accuracy: 1.0000
Epoch 114/200
28/28 [==============================] - 1s 30ms/step - loss: 1.3790e-05 - accuracy: 1.0000
Epoch 115/200
28/28 [==============================] - 1s 31ms/step - loss: 1.3355e-05 - accuracy: 1.0000
Epoch 116/200
28/28 [==============================] - 1s 31ms/step - loss: 1.2889e-05 - accuracy: 1.0000
Epoch 117/200
28/28 [==============================] - 1s 31ms/step - loss: 1.2479e-05 - accuracy: 1.0000
Epoch 118/200
28/28 [==============================] - 1s 27

...

28/28 [==============================] - 1s 30ms/step - loss: 2.7813e-05 - accuracy: 1.0000
Epoch 63/200
28/28 [==============================] - 1s 31ms/step - loss: 2.6579e-05 - accuracy: 1.0000
Epoch 64/200
28/28 [==============================] - 1s 32ms/step - loss: 2.4382e-05 - accuracy: 1.0000
Epoch 65/200
28/28 [==============================] - 1s 31ms/step - loss: 2.3110e-05 - accuracy: 1.0000
Epoch 66/200
28/28 [==============================] - 1s 33ms/step - loss: 2.1630e-05 - accuracy: 1.0000
Epoch 67/200
28/28 [==============================] - 1s 32ms/step - loss: 2.0581e-05 - accuracy: 1.0000
Epoch 68/200
28/28 [==============================] - 1s 33ms/step - loss: 1.9591e-05 - accuracy: 1.0000
Epoch 69/200
28/28 [==============================] - 1s 29ms/step - loss: 1.8452e-05 - accuracy: 1.0000
Epoch 70/200
28/28 [==============================] - 1s 35ms/step - loss: 1.7643e-05 - accuracy: 1.0000
Epoch 71/200
28/28 [==============================] - 1s 31ms/step -

28/28 [==============================] - 1s 33ms/step - loss: 1.6616e-06 - accuracy: 1.0000
Epoch 140/200
28/28 [==============================] - 1s 32ms/step - loss: 1.6268e-06 - accuracy: 1.0000
Epoch 141/200
28/28 [==============================] - 1s 31ms/step - loss: 1.5864e-06 - accuracy: 1.0000
Epoch 142/200
28/28 [==============================] - 1s 31ms/step - loss: 1.5555e-06 - accuracy: 1.0000
Epoch 143/200
28/28 [==============================] - 1s 32ms/step - loss: 1.5177e-06 - accuracy: 1.0000
Epoch 144/200
28/28 [==============================] - 1s 31ms/step - loss: 1.4877e-06 - accuracy: 1.0000
Epoch 145/200
28/28 [==============================] - 1s 30ms/step - loss: 1.4524e-06 - accuracy: 1.0000
Epoch 146/200
28/28 [==============================] - 1s 33ms/step - loss: 1.4201e-06 - accuracy: 1.0000
Epoch 147/200
28/28 [==============================] - 1s 31ms/step - loss: 1.3863e-06 - accuracy: 1.0000
Epoch 148/200
28/28 [==============================] - 1s 31

Epoch 15/200
28/28 [==============================] - 1s 31ms/step - loss: 0.0962 - accuracy: 0.9739
Epoch 16/200
28/28 [==============================] - 1s 33ms/step - loss: 0.0960 - accuracy: 0.9659
Epoch 17/200
28/28 [==============================] - 1s 32ms/step - loss: 0.0376 - accuracy: 0.9932
Epoch 18/200
28/28 [==============================] - 1s 30ms/step - loss: 0.0280 - accuracy: 0.9932
Epoch 19/200
28/28 [==============================] - 1s 31ms/step - loss: 0.0111 - accuracy: 0.9989
Epoch 20/200
28/28 [==============================] - 1s 30ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 21/200
28/28 [==============================] - 1s 29ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 22/200
28/28 [==============================] - 1s 29ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 23/200
28/28 [==============================] - 1s 29ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 24/200
28/28 [==============================] - 1s 29ms/step - loss: 0.0011 - accurac

28/28 [==============================] - 1s 32ms/step - loss: 2.9963e-05 - accuracy: 1.0000
Epoch 94/200
28/28 [==============================] - 1s 30ms/step - loss: 2.9174e-05 - accuracy: 1.0000
Epoch 95/200
28/28 [==============================] - 1s 33ms/step - loss: 2.8364e-05 - accuracy: 1.0000
Epoch 96/200
28/28 [==============================] - 1s 32ms/step - loss: 2.7515e-05 - accuracy: 1.0000
Epoch 97/200
28/28 [==============================] - 1s 30ms/step - loss: 2.6715e-05 - accuracy: 1.0000
Epoch 98/200
28/28 [==============================] - 1s 32ms/step - loss: 2.6038e-05 - accuracy: 1.0000
Epoch 99/200
28/28 [==============================] - 1s 32ms/step - loss: 2.5334e-05 - accuracy: 1.0000
Epoch 100/200
28/28 [==============================] - 1s 31ms/step - loss: 2.4695e-05 - accuracy: 1.0000
Epoch 101/200
28/28 [==============================] - 1s 30ms/step - loss: 2.4050e-05 - accuracy: 1.0000
Epoch 102/200
28/28 [==============================] - 1s 32ms/ste

28/28 [==============================] - 1s 29ms/step - loss: 4.9855e-06 - accuracy: 1.0000
Epoch 171/200
28/28 [==============================] - 1s 30ms/step - loss: 4.8919e-06 - accuracy: 1.0000
Epoch 172/200
28/28 [==============================] - 1s 31ms/step - loss: 4.8122e-06 - accuracy: 1.0000
Epoch 173/200
28/28 [==============================] - 1s 30ms/step - loss: 4.6837e-06 - accuracy: 1.0000
Epoch 174/200
28/28 [==============================] - 1s 31ms/step - loss: 4.6025e-06 - accuracy: 1.0000
Epoch 175/200
28/28 [==============================] - 1s 34ms/step - loss: 4.5089e-06 - accuracy: 1.0000
Epoch 176/200
28/28 [==============================] - 1s 33ms/step - loss: 4.4215e-06 - accuracy: 1.0000
Epoch 177/200
28/28 [==============================] - 1s 33ms/step - loss: 4.3340e-06 - accuracy: 1.0000
Epoch 178/200
28/28 [==============================] - 1s 31ms/step - loss: 4.2513e-06 - accuracy: 1.0000
Epoch 179/200
28/28 [==============================] - 1s 31

Best results <br>
Accuracy of  Valence 51.75 <br>
Accuracy of  Arousal 51.25 <br>
Accuracy of  Dominance 59.25 <br>
Accuracy of  Liking 59.25 <br>